In [1]:
import os
os.environ.get('JAVA_HOME')
os.environ.get('SPARK_HOME')

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Starting with Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

23/06/13 08:55:14 WARN Utils: Your hostname, Viniciuss-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.16.0.149 instead (on interface en0)
23/06/13 08:55:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/13 08:55:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
## Simple example, how to create a Spark DataFrame

data = [('Will', '23'), ('Bill', '32')]
colNames = ['name', 'age']
df = spark.createDataFrame(data, colNames)
df.show()

+----+---+
|name|age|
+----+---+
|Will| 23|
|Bill| 32|
+----+---+



In [34]:
# Using the Brazil Receita Federal data about business registration number (CNPJ);

business = spark.read.csv('data/empresas/', sep=';', inferSchema=True)

business_col_names = ['root_cnpj', 'company_legal_name', 'legal_nature', 'responsible_qualification', \
    'company_share_capital', 'business_size', 'federative_entity_responsible']

for index, col_name in enumerate(business_col_names):
    business = business.withColumnRenamed(f"_c{index}", col_name)
    
business.columns

['root_cnpj',
 'company_legal_name',
 'legal_nature',
 'responsible_qualification',
 'company_share_capital',
 'business_size',
 'federative_entity_responsible']

In [35]:
establishments = spark.read.csv('data/estabelecimentos/', sep=';', inferSchema=True)

estab_col_names = ['root_cnpj', 'order_cnpj', 'cnpj_dv', 'id_hq_or_branch', 'company_fantasy_name',\
    'registration_status', 'date_registration_status', 'reason_registration_status', 'city_outside_name',\
        'country', 'activity_start_date', 'main_cnae', 'secundary_cnae',\
            'street_type', 'street', 'number', 'complement', 'neighborhood', 'zip_code', 'state_code', \
                'city', 'ddd_1', 'phone_1', 'ddd_2', 'phone_2', 'ddd_fax', 'fax', \
                    'email', 'special_situation', 'date_special_situation']

for index, col_name in enumerate(estab_col_names):
    establishments = establishments.withColumnRenamed(f"_c{index}", col_name)
    
establishments.columns

['root_cnpj',
 'order_cnpj',
 'cnpj_dv',
 'id_hq_or_branch',
 'company_fantasy_name',
 'registration_status',
 'date_registration_status',
 'reason_registration_status',
 'city_outside_name',
 'country',
 'activity_start_date',
 'main_cnae',
 'secundary_cnae',
 'street_type',
 'street',
 'number',
 'complement',
 'neighborhood',
 'zip_code',
 'state_code',
 'city',
 'ddd_1',
 'phone_1',
 'ddd_2',
 'phone_2',
 'ddd_fax',
 'fax',
 'email',
 'special_situation',
 'date_special_situation']

In [14]:
partners = s

partners_col_names = ['root_cnpj', 'partner_id', 'partner_name_or_legal_name', \
    'cnpj_cpf_partner', 'partner_qualificstion', 'entry_date_partnership', 'country', 'legal_representative',\
        'representative_name', 'legal_representative_qualification', 'age_group']